--- Day 23: Amphipod ---

A group of amphipods notice your fancy submarine and flag you down. "With such an impressive shell," one amphipod says, "surely you can help us with a question that has stumped our best scientists."

They go on to explain that a group of timid, stubborn amphipods live in a nearby burrow. Four types of amphipods live there: Amber (A), Bronze (B), Copper (C), and Desert (D). They live in a burrow that consists of a hallway and four side rooms. The side rooms are initially full of amphipods, and the hallway is initially empty.

They give you a diagram of the situation (your puzzle input), including locations of each amphipod (A, B, C, or D, each of which is occupying an otherwise open space), walls (#), and open space (.).

For example:

```
#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
```

The amphipods would like a method to organize every amphipod into side rooms so that each side room contains one type of amphipod and the types are sorted A-D going left to right, like this:

```
#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
```  

Amphipods can move up, down, left, or right so long as they are moving into an unoccupied open space. Each type of amphipod requires a different amount of energy to move one step: Amber amphipods require 1 energy per step, Bronze amphipods require 10 energy, Copper amphipods require 100, and Desert ones require 1000. The amphipods would like you to find a way to organize the amphipods that requires the least total energy.

However, because they are timid and stubborn, the amphipods have some extra rules:

-    Amphipods will never stop on the space immediately outside any room. They can move into that space so long as they immediately continue moving. (Specifically, this refers to the four open spaces in the hallway that are directly above an amphipod starting position.)
-    Amphipods will never move from the hallway into a room unless that room is their destination room and that room contains no amphipods which do not also have that room as their own destination. If an amphipod's starting room is not its destination room, it can stay in that room until it leaves the room. (For example, an Amber amphipod will not move from the hallway into the right three rooms, and will only move into the leftmost room if that room is empty or if it only contains other Amber amphipods.)
-    Once an amphipod stops moving in the hallway, it will stay in that spot until it can move into a room. (That is, once any amphipod starts moving, any other amphipods currently in the hallway are locked in place and will not move again until they can move fully into a room.)

In the above example, the amphipods can be organized using a minimum of 12521 energy. One way to do this is shown below.

Starting configuration:

```
#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
```

One Bronze amphipod moves into the hallway, taking 4 steps and using 40 energy:

```
#############
#...B.......#
###B#C#.#D###
  #A#D#C#A#
  #########
```  

The only Copper amphipod not in its side room moves there, taking 4 steps and using 400 energy:

```
#############
#...B.......#
###B#.#C#D###
  #A#D#C#A#
  #########
```  

A Desert amphipod moves out of the way, taking 3 steps and using 3000 energy, and then the Bronze amphipod takes its place, taking 3 steps and using 30 energy:

```
#############
#.....D.....#
###B#.#C#D###
  #A#B#C#A#
  #########
```  

The leftmost Bronze amphipod moves to its room using 40 energy:

```
#############
#.....D.....#
###.#B#C#D###
  #A#B#C#A#
  #########
```  

Both amphipods in the rightmost room move into the hallway, using 2003 energy in total:

```
#############
#.....D.D.A.#
###.#B#C#.###
  #A#B#C#.#
  #########
```  

Both Desert amphipods move into the rightmost room using 7000 energy:

```
#############
#.........A.#
###.#B#C#D###
  #A#B#C#D#
  #########
```  

Finally, the last Amber amphipod moves into its room, using 8 energy:

```
#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
```  

What is the least energy required to organize the amphipods?


In [ ]:
type Room = A | B | C | D
type Position = Room of Room * int * bool | Hall of int
let Multiplier = 
    Map.empty
        .Add( A, 1 )
        .Add( B, 10 )
        .Add( C, 100 )
        .Add( D, 1000 )
let RoomPosition = 
    Map.empty
        .Add( A, 2)
        .Add( B, 4 )
        .Add( C, 6 )
        .Add( D, 8 )

type AmphibianState = Room * Position
let testInput: AmphibianState array = 
    [|
        (A, Room( A, 0, false))
        (B, Room( A, 1, false))
        (D, Room( B, 0, false))
        (C, Room( B, 1, false))
        (C, Room( C, 0, false))
        (B, Room( C, 1, false))
        (A, Room( D, 0, false))
        (D, Room( D, 1, false))
    |]

testInput

index Item1 Item2 0 A Item1 Item2 Item3 A 0 False 1 B Item1 Item2 Item3 A 1 False 2 D Item1 Item2 Item3 B 0 False 3 C Item1 Item2 Item3 B 1 False 4 C Item1 Item2 Item3 C 0 False 5 B Item1 Item2 Item3 C 1 False 6 A Item1 Item2 Item3 D 0 False 7 D Item1 Item2 Item3 D 1 False

In [ ]:
let stateToOccupancy (roomSize: int) (state: seq<AmphibianState> ) =
    let hallway = Array.create 11 -1

    let mutable rooms = 
        Map.empty
            .Add(A, (0, Array.create roomSize -1 ) )
            .Add(B, (0, Array.create roomSize -1 ) )
            .Add(C, (0, Array.create roomSize -1 ) )
            .Add(D, (0, Array.create roomSize -1 ) )
            
    state 
    |> Seq.iteri ( fun index (_, position) ->
        match position with
        | Room (room, pos, final) -> 
            let (size, entries) = rooms[room];
            entries[pos] <- index
            rooms <- rooms.Add( room, (size + 1, entries) )
        | Hall pos -> 
            hallway[pos] <- index
    )

    (hallway, rooms)

stateToOccupancy 2 [| (A, Hall(3)); (B, Room(C, 1, false)); (B, Room(C, 0, false)) |]

Item1,Item2
"[ -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1 ]","[ { [A, (0, System.Int32[])]: Key: A, Value: ( 0, [ -1, -1 ] ) }, { [B, (0, System.Int32[])]: Key: B, Value: ( 0, [ -1, -1 ] ) }, { [C, (2, System.Int32[])]: Key: C, Value: ( 2, [ 2, 1 ] ) }, { [D, (0, System.Int32[])]: Key: D, Value: ( 0, [ -1, -1 ] ) } ]"


In [ ]:
let getPossibleMoves (amphibianIndex: int) (state: AmphibianState array) (hallway: int[]) (rooms: Map<Room, int * int array>) =
    let (amphibian, position) = state[amphibianIndex];
    let canGoHome =
        let (size, places) = rooms[amphibian]
        places |> Array.forall( fun i ->
            if i = -1 then
                true
            else 
                let ( t, _) = state[i]
                t = amphibian
        )
    
    let freeHallway (fromIndex:int) (toIndex:int) = 
        if fromIndex = toIndex then
            [||]
        elif fromIndex < toIndex then
            hallway[(fromIndex+1) .. toIndex]
        else
            hallway[toIndex .. (fromIndex-1)]
        |> 
        Array.forall( (=) -1 )


    match position with
    // from hall try going home, if the home is free and path is open
    | Hall pos when canGoHome && ( freeHallway pos RoomPosition[amphibian] ) ->
        let (occupancy, places) = rooms[amphibian]
        let cost = Math.Abs( RoomPosition[amphibian] - pos ) + places.Length - occupancy
        [ ( Room(amphibian, occupancy, true ), cost * Multiplier[amphibian] ) ]
    // otherwise we cannot move from hall
    | Hall _ -> 
        []
    // when in room and done, no move
    | Room (room, pos, isDone) when isDone = true ->
        []
    // when in room, try going home, if the home is free and the path is open
    | Room (room, pos, isDone) when canGoHome && ( freeHallway RoomPosition[room] RoomPosition[amphibian] ) ->
        if pos < (fst rooms[room]) - 1 then
            // we are blocked by other
            []
        elif room = amphibian then
            // if we're already in the room, we just mark amphibian as done
            [ ( Room(room, pos, true)), 0 ]            
        else 
            // go directly from home to home
            let (occupancy, places) = rooms[amphibian]
            let cost = Math.Abs( RoomPosition[amphibian] - RoomPosition[room] ) + places.Length - occupancy + places.Length - pos // we assume all places arrays have same length
            [ ( Room(amphibian, occupancy, true ), cost * Multiplier[amphibian] )]
    | Room (room, pos, _) ->
        let currentPosition = RoomPosition[room]
        let (occupancy, places) = rooms[room]        
        if pos < occupancy - 1 then
            // someone else is above, we cannot move
            []
        else
            [0; 1; 3; 5; 7; 9; 10]
            |> List.filter (fun target -> freeHallway currentPosition target )
            |> List.map( fun target ->
                let cost = Math.Abs( currentPosition - target ) + places.Length - pos
                ( Hall(target), cost * Multiplier[amphibian] )
            )
    | _ -> []
        
[|
    (A, Room( A, 0, false))
    // (B, Room( A, 1, false))
    (D, Room( B, 0, false))
    (C, Room( B, 1, false))
    (C, Room( C, 0, false))
    (B, Room( C, 1, false))
    (A, Room( D, 0, false))
    (D, Room( D, 1, false))|] 
|> ( fun state -> 
    let (hallway, rooms) = stateToOccupancy 2 state
    getPossibleMoves 5 state hallway rooms
)


(empty)

In [ ]:
let findSolution (depth: int) (input: AmphibianState array): (int option) =
    let mutable cache = Map.empty

    let rec doSteps (state: AmphibianState array) =
        if cache.ContainsKey state then
            cache[state]
        elif (Array.forall (
                    fun (amphibian, amphibianState) ->
                    match amphibianState with
                        | Room (_, _, true) -> true
                        | _ -> false
                )  
                state
        )
        then
            cache <- cache.Add(state, Some (0, []))
            Some (0, [])
        else
            // printfn "%i %A" depth state
            let (hallway, rooms) = stateToOccupancy depth state
            let result = 
                seq {
                    for i=0 to input.Length-1 do
                        getPossibleMoves i state hallway rooms
                        |> Seq.map ( fun (newPosition, cost) -> 
                            (i, newPosition, cost)
                        )
                }
                |> Seq.concat
//                |> Seq.sortBy ( fun (i, newPos, cost) -> cost )
                |> Seq.map( fun (i, newPos, cost) -> 
                    let newState = Array.copy state
                    let (amphibian, oldPos) = state[i]
                    newState[i] <- (amphibian, newPos)
                    match (doSteps newState) with
                    | Some (remainingCost, moves) ->
                        Some (cost + remainingCost, (amphibian, oldPos, newPos, cost) :: moves )
                    | None -> None
                )
                |> Seq.choose id 
                |> Array.ofSeq
                |> 
                ( fun possibilities ->
                    if possibilities.Length = 0 then
                        None
                    else 
                        possibilities |> Seq.minBy fst |> Some
                )
            cache <- cache.Add( 
                state,
                result
            )
            result

    let res = doSteps input
    printfn "Cache size %i" (cache |> Map.keys |> Seq.length )
    if res.IsSome then
        printfn "%A" res
        Some( fst (res.Value) )
    else
        None        

testInput |> findSolution 2 // 12521

Cache size 75388
Some
  (12521,
   [(B, Room (C, 1, false), Hall 3, 40);
    (C, Room (C, 0, false), Room (C, 0, true), 0);
    (C, Room (B, 1, false), Room (C, 1, true), 400);
    (D, Room (B, 0, false), Hall 5, 3000); (B, Hall 3, Room (B, 0, true), 30);
    (B, Room (A, 1, false), Room (B, 1, true), 40);
    (A, Room (A, 0, false), Room (A, 0, true), 0);
    (D, Room (D, 1, false), Hall 7, 2000); (A, Room (D, 0, false), Hall 9, 3);
    (D, Hall 7, Room (D, 0, true), 3000); (D, Hall 5, Room (D, 1, true), 4000);
    (A, Hall 9, Room (A, 1, true), 8)])


Value
12521


In [ ]:
let task = [|
    (D, Room( A, 0, false))
    (B, Room( A, 1, false))
    (C, Room( B, 0, false))
    (B, Room( B, 1, false))
    (A, Room( C, 0, false))
    (D, Room( C, 1, false))
    (C, Room( D, 0, false))
    (A, Room( D, 1, false))
|]
task |> findSolution 2  // 15412

Cache size 144234
Some
  (15412,
   [(D, Room (C, 1, false), Hall 7, 2000); (A, Room (C, 0, false), Hall 1, 7);
    (B, Room (B, 1, false), Hall 3, 20);
    (C, Room (B, 0, false), Room (C, 0, true), 600);
    (B, Hall 3, Room (B, 0, true), 30);
    (B, Room (A, 1, false), Room (B, 1, true), 40);
    (D, Room (A, 0, false), Hall 3, 3000); (A, Hall 1, Room (A, 0, true), 3);
    (A, Room (D, 1, false), Hall 10, 3); (C, Room (D, 0, false), Hall 9, 300);
    (D, Hall 7, Room (D, 0, true), 3000); (D, Hall 3, Room (D, 1, true), 6000);
    (C, Hall 9, Room (C, 1, true), 400); (A, Hall 10, Room (A, 1, true), 9)])


Value
15412


--- Part Two ---

As you prepare to give the amphipods your solution, you notice that the diagram they handed you was actually folded up. As you unfold it, you discover an extra part of the diagram.

Between the first and second lines of text that contain amphipod starting positions, insert the following lines:

  #D#C#B#A#
  #D#B#A#C#

So, the above example now becomes:

#############
#...........#
###B#C#B#D###
  #D#C#B#A#
  #D#B#A#C#
  #A#D#C#A#
  #########

The amphipods still want to be organized into rooms similar to before:

#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #A#B#C#D#
  #A#B#C#D#
  #########

In this updated example, the least energy required to organize these amphipods is 44169:

#############
#...........#
###B#C#B#D###
  #D#C#B#A#
  #D#B#A#C#
  #A#D#C#A#
  #########

#############
#..........D#
###B#C#B#.###
  #D#C#B#A#
  #D#B#A#C#
  #A#D#C#A#
  #########

#############
#A.........D#
###B#C#B#.###
  #D#C#B#.#
  #D#B#A#C#
  #A#D#C#A#
  #########

#############
#A........BD#
###B#C#.#.###
  #D#C#B#.#
  #D#B#A#C#
  #A#D#C#A#
  #########

#############
#A......B.BD#
###B#C#.#.###
  #D#C#.#.#
  #D#B#A#C#
  #A#D#C#A#
  #########

#############
#AA.....B.BD#
###B#C#.#.###
  #D#C#.#.#
  #D#B#.#C#
  #A#D#C#A#
  #########

#############
#AA.....B.BD#
###B#.#.#.###
  #D#C#.#.#
  #D#B#C#C#
  #A#D#C#A#
  #########

#############
#AA.....B.BD#
###B#.#.#.###
  #D#.#C#.#
  #D#B#C#C#
  #A#D#C#A#
  #########

#############
#AA...B.B.BD#
###B#.#.#.###
  #D#.#C#.#
  #D#.#C#C#
  #A#D#C#A#
  #########

#############
#AA.D.B.B.BD#
###B#.#.#.###
  #D#.#C#.#
  #D#.#C#C#
  #A#.#C#A#
  #########

#############
#AA.D...B.BD#
###B#.#.#.###
  #D#.#C#.#
  #D#.#C#C#
  #A#B#C#A#
  #########

#############
#AA.D.....BD#
###B#.#.#.###
  #D#.#C#.#
  #D#B#C#C#
  #A#B#C#A#
  #########

#############
#AA.D......D#
###B#.#.#.###
  #D#B#C#.#
  #D#B#C#C#
  #A#B#C#A#
  #########

#############
#AA.D......D#
###B#.#C#.###
  #D#B#C#.#
  #D#B#C#.#
  #A#B#C#A#
  #########

#############
#AA.D.....AD#
###B#.#C#.###
  #D#B#C#.#
  #D#B#C#.#
  #A#B#C#.#
  #########

#############
#AA.......AD#
###B#.#C#.###
  #D#B#C#.#
  #D#B#C#.#
  #A#B#C#D#
  #########

#############
#AA.......AD#
###.#B#C#.###
  #D#B#C#.#
  #D#B#C#.#
  #A#B#C#D#
  #########

#############
#AA.......AD#
###.#B#C#.###
  #.#B#C#.#
  #D#B#C#D#
  #A#B#C#D#
  #########

#############
#AA.D.....AD#
###.#B#C#.###
  #.#B#C#.#
  #.#B#C#D#
  #A#B#C#D#
  #########

#############
#A..D.....AD#
###.#B#C#.###
  #.#B#C#.#
  #A#B#C#D#
  #A#B#C#D#
  #########

#############
#...D.....AD#
###.#B#C#.###
  #A#B#C#.#
  #A#B#C#D#
  #A#B#C#D#
  #########

#############
#.........AD#
###.#B#C#.###
  #A#B#C#D#
  #A#B#C#D#
  #A#B#C#D#
  #########

#############
#..........D#
###A#B#C#.###
  #A#B#C#D#
  #A#B#C#D#
  #A#B#C#D#
  #########

#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #A#B#C#D#
  #A#B#C#D#
  #########

Using the initial configuration from the full diagram, what is the least energy required to organize the amphipods?


In [ ]:
let testInput2: AmphibianState array = 
    [|
        (A, Room( A, 0, false))
        (D, Room( A, 1, false))
        (D, Room( A, 2, false))
        (B, Room( A, 3, false))

        (D, Room( B, 0, false))
        (B, Room( B, 1, false))
        (C, Room( B, 2, false))
        (C, Room( B, 3, false))

        (C, Room( C, 0, false))
        (A, Room( C, 1, false))
        (B, Room( C, 2, false))
        (B, Room( C, 3, false))

        (A, Room( D, 0, false))
        (C, Room( D, 1, false))
        (A, Room( D, 2, false))
        (D, Room( D, 3, false))
    |]

testInput2 |> findSolution 4  // 44169

Cache size 586424
Some
  (44169,
   [(D, Room (D, 3, false), Hall 10, 3000); (B, Room (C, 3, false), Hall 9, 40);
    (A, Room (D, 2, false), Hall 0, 10); (B, Room (C, 2, false), Hall 7, 30);
    (A, Room (C, 1, false), Hall 1, 8);
    (C, Room (C, 0, false), Room (C, 0, true), 0);
    (C, Room (B, 3, false), Room (C, 1, true), 600);
    (C, Room (B, 2, false), Room (C, 2, true), 600);
    (B, Room (B, 1, false), Hall 5, 40); (D, Room (B, 0, false), Hall 3, 5000);
    (B, Hall 5, Room (B, 0, true), 50); (B, Hall 7, Room (B, 1, true), 60);
    (B, Hall 9, Room (B, 2, true), 70);
    (C, Room (D, 1, false), Room (C, 3, true), 600);
    (A, Room (D, 0, false), Hall 9, 5); (D, Hall 3, Room (D, 0, true), 9000);
    (B, Room (A, 3, false), Room (B, 3, true), 40);
    (D, Room (A, 2, false), Room (D, 1, true), 11000);
    (D, Room (A, 1, false), Room (D, 2, true), 11000);
    (A, Room (A, 0, false), Room (A, 0, true), 0);
    (A, Hall 1, Room (A, 1, true), 4); (A, Hall 9, Room (A, 2, true), 9

Value
44169


In [ ]:
let task2: AmphibianState array = 
    [|
        (D, Room( A, 0, false))
        (D, Room( A, 1, false))
        (D, Room( A, 2, false))
        (B, Room( A, 3, false))

        (C, Room( B, 0, false))
        (B, Room( B, 1, false))
        (C, Room( B, 2, false))
        (B, Room( B, 3, false))

        (A, Room( C, 0, false))
        (A, Room( C, 1, false))
        (B, Room( C, 2, false))
        (D, Room( C, 3, false))

        (C, Room( D, 0, false))
        (C, Room( D, 1, false))
        (A, Room( D, 2, false))
        (A, Room( D, 3, false))
    |]

task2 |> findSolution 4  // 52358

Cache size 276309
Some
  (52358,
   [(B, Room (A, 3, false), Hall 0, 30); (D, Room (A, 2, false), Hall 1, 3000);
    (D, Room (A, 1, false), Hall 10, 11000);
    (D, Room (A, 0, false), Hall 9, 11000);
    (A, Room (D, 3, false), Room (A, 0, true), 11);
    (A, Room (D, 2, false), Room (A, 1, true), 11);
    (C, Room (D, 1, false), Hall 3, 800); (C, Room (D, 0, false), Hall 5, 700);
    (D, Hall 9, Room (D, 0, true), 5000); (D, Hall 10, Room (D, 1, true), 5000);
    (D, Room (C, 3, false), Room (D, 2, true), 5000);
    (B, Room (C, 2, false), Hall 10, 60); (A, Room (C, 1, false), Hall 9, 6);
    (A, Room (C, 0, false), Hall 7, 5); (C, Hall 5, Room (C, 0, true), 500);
    (C, Hall 3, Room (C, 1, true), 600); (A, Hall 7, Room (A, 2, true), 7);
    (D, Hall 1, Room (D, 3, true), 8000); (B, Room (B, 3, false), Hall 1, 40);
    (C, Room (B, 2, false), Room (C, 2, true), 600);
    (B, Room (B, 1, false), Hall 3, 40);
    (C, Room (B, 0, false), Room (C, 3, true), 700);
    (B, Hall 3, Room (

Value
52358
